## Text Mining in Social Media - TF-IDF

*En este notebook se detalla paso a paso todo el proceso de extracción de tuits de usuarios, su inserción en un DataFrame y el entrenamiento, etiquetado y evaluación de un modelo capaz de predecir el sexo y la variedad de español de un usuario dados 100 de sus tuits.*

**1- Generamos dos diccionarios llamados ids_train e ids_test que contienen los ids de usuarios clasificados por variedad y sexo. Usamos para ello la función generar_dicc_ids**

Para ello abrimos el fichero truth.txt, que contiene estos campos en forma de csv separados por los caracteres ':::'.

In [1]:
def generar_dicc_ids(particion):
    # particion: 'train' o 'test'
    if particion=='train':
        f = open('./PAN-AP/training/truth.txt')
    else:
        f = open('./PAN-AP/test/truth.txt')
    
    contador=0
    sexo={'male':0,'female':0}
    variedades={'colombia':0,'argentina':0,'spain':0,'venezuela':0,'peru':0,'chile':0,'mexico':0}

    ids={'colombia':{'male':[],'female':[]},'argentina':{'male':[],'female':[]},'spain':{'male':[],'female':[]}, \
     'venezuela':{'male':[],'female':[]},'peru':{'male':[],'female':[]},'chile':{'male':[],'female':[]}, \
     'mexico':{'male':[],'female':[]}}

    for line in f:
        line=line.rstrip()
        if len(line)==0:
            continue
        lista=line.split(':::')

        ids[lista[2]][lista[1]].append(lista[0])

        sexo[lista[1]] += 1
        variedades[lista[2]] += 1
        contador+=1

    print(sexo)
    print(variedades)
    print(contador)

    return ids

ids_train = generar_dicc_ids('train')
ids_test  = generar_dicc_ids('test')


{'male': 1400, 'female': 1400}
{'colombia': 400, 'argentina': 400, 'spain': 400, 'venezuela': 400, 'peru': 400, 'chile': 400, 'mexico': 400}
2800
{'male': 700, 'female': 700}
{'colombia': 200, 'argentina': 200, 'spain': 200, 'venezuela': 200, 'peru': 200, 'chile': 200, 'mexico': 200}
1400


**Ejemplo de los diccionarios generados**

In [2]:
ids_train['spain']['male'][0:10]

['ef3588c9462713023145ae3c12c85614',
 '17e06a4ef15eaa851242465edc5328bd',
 'cfa38327f7699d48daaaaf4278a1354a',
 'e85fd98dfc6743bce7b274fbcac69f24',
 '826e3b4e72bfb6f9cdfc1a6995be10e5',
 'bf149d41c5e685054a57fb38d964735d',
 '79e9e99239fe662190978d62ebc4c24d',
 '2d56280a969d9dbcc4b32b21bc0a9b02',
 'b56c7a73e3bb2eb5436a3642b1fb70ee',
 '17786b7a4a1a31775af8ae786b4e4711']

**2- Definimos dos funciones:**

**flatstring:** recibe una lista de strings y los concatena devolviendo un único string. Añade un espacio en blanco entre uno y otro para que estén separados.

**leer_tuits:** recibe el id del usuario y a qué partición pertenece ('train' o 'test'). Devuelve los 100 tuits del usuario concatenados en un único string.



In [3]:
def flatstring(lista):
    s=""
    for item in lista:
        s+= " " + item
    return s

def leer_tuits(id,particion):
    if particion == 'train':
        s='./PAN-AP/training/' + id + '.xml'
    else:
        s='./PAN-AP/test/' + id + '.xml'
    f=open(s)
    tuits=list()
    for line in f:
        line=line.rstrip()
        if line.find('<document><![CDATA[') != -1:
            line=line[21:-14]
            tuits.append(line)
    return flatstring(tuits)


**3- Definimos la función generar_dataframe que devuelve un DataFrame de pandas con las columnas tuits, sexo y variedad. En cada fila de la columna tuits, se encuentran todos los tuits de un usuario concatenados en un único string.**

In [4]:
import pandas as pd

def generar_dataframe(ids,particion):
    # ids: ids_train o ids_test
    
    df = pd.DataFrame(columns = ['tuits','sexo','variedad'])

    i=0 # Contador del número de filas. Cada fila será un usuario.
    variedades_lista = ['argentina', 'chile', 'colombia', 'mexico', 'peru', 'spain', 'venezuela']
    sexo_lista = ['female','male']

    for variedad in variedades_lista:
        v=0 # contador del número de usuarios de una variedad
        print(variedad)
        
        for sexo in sexo_lista:
            s=0 # contador del número de usuarios de un sexo
            
            for id in ids[variedad][sexo]:
                t = leer_tuits(id,particion)
                df.loc[i] = [t,sexo,variedad]
                i+=1
                v+=1
                s+=1
            
            print("\t",sexo,s)
        
        print(v)
    
    return df


Generamos los DataFrames train y test.

In [5]:
train = generar_dataframe(ids_train, 'train')

argentina
	 female 200
	 male 200
400
chile
	 female 200
	 male 200
400
colombia
	 female 200
	 male 200
400
mexico
	 female 200
	 male 200
400
peru
	 female 200
	 male 200
400
spain
	 female 200
	 male 200
400
venezuela
	 female 200
	 male 200
400


In [6]:
test  = generar_dataframe(ids_test, 'test')

argentina
	 female 100
	 male 100
200
chile
	 female 100
	 male 100
200
colombia
	 female 100
	 male 100
200
mexico
	 female 100
	 male 100
200
peru
	 female 100
	 male 100
200
spain
	 female 100
	 male 100
200
venezuela
	 female 100
	 male 100
200


Ejemplo del DataFrame train

In [7]:
train.sample(n=10)

,tuits,sexo,variedad
1840,Sigamos apoyando con el HT #MissUniverse #Per...,male,peru
761,Cuando no tuiteo se escapan los fologüers @ea...,male,chile
854,@GloriaTrevi me encanta el color del pastel l...,female,colombia
2361,@MikyGonka @Cinesa LA LA Land es 💩 ve a ver @...,male,spain
1878,@Nadia__LV @gabrielatuesta jajaja calla calla...,male,peru
2480,Chrystal Moselle y titulado 'Bring it' que ca...,female,venezuela
1425,"Amigos y hermanos mexicanos en Niza, contacte...",male,mexico
145,Q lindo q escuchar la lluvia me encanta🙈😍 👌 @...,female,argentina
1346,El otro día compre un juego de cuchillos y un...,female,mexico
912,@ARZAHE1 @EliasBarrios3 @victor_admon @wilis_...,female,colombia


In [8]:
train.describe()

,tuits,sexo,variedad
count,2800,2800,2800
unique,2800,2,7
top,@vhmolinac @ciroalonp No sé porque se siente ...,male,mexico
freq,1,1400,400


**4- Para preparar los datos de entrenamiento y de test, usamos la función TfidfVectorizer que crea una bolsa de palabras ponderada según la importancia de una palabra que pertenece a un usuario dentro de un conjunto de usuarios.**

**Parámetros:** 
- **tokenizer**: usamos TweetTokenizer que elimina caracteres repetidos más de 3 veces, elimina mayúsculas y elimina espacios en blanco.
- **stop_words**: descartamos las stop words mediante la librería stop_words de Python.
- **max_features**: obtenemos únicamente las 20.000 palabras con mayor frecuencia.

In [9]:
from nltk.tokenize import TweetTokenizer
from stop_words import get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer

tokenizador = TweetTokenizer(reduce_len=True, preserve_case=False, strip_handles=True)
stop_words = get_stop_words('es')

vec = TfidfVectorizer(tokenizer=tokenizador.tokenize,max_features=20000,stop_words=stop_words)

**Generamos las matrices x_train (2800x20000) y x_test (1400x20000) que son las matrices X de entrada al modelo.**

**Usamos los métodos fit_transform para la matriz de train y transform para la matriz de test.**

In [10]:
x_train = vec.fit_transform(train['tuits'])
x_test  = vec.transform(test['tuits'])

x_train, x_test

(<2800x20000 sparse matrix of type '<class 'numpy.float64'>'
 	with 1072302 stored elements in Compressed Sparse Row format>,
 <1400x20000 sparse matrix of type '<class 'numpy.float64'>'
 	with 537416 stored elements in Compressed Sparse Row format>)

**5- SEXO: clasificador, predicción y evaluación.**

**El mejor clasificador encontrado se basa en un modelo RandomForest con 500 árboles**

**Creación del clasificador:** usamos el método fit de la función RandomForestClassifier con n=500

In [11]:
from sklearn.ensemble import RandomForestClassifier

clf_gender = RandomForestClassifier(500).fit(x_train, train['sexo'])

**Predicción:** usamos el método predict de RandomForestClassifier

In [12]:
predicted_gender = clf_gender.predict(x_test)

Ejemplo de la predicción

In [13]:
predicted_gender[0:10]

array(['female', 'male', 'female', 'male', 'female', 'male', 'female',
       'female', 'female', 'female'], dtype=object)

**Evaluación:** usamos la función classification_report de la librería metrics de sklearn

In [14]:
from sklearn import metrics

print(metrics.classification_report(test.sexo, predicted_gender))

             precision    recall  f1-score   support

     female       0.75      0.71      0.73       700
       male       0.72      0.76      0.74       700

avg / total       0.74      0.73      0.73      1400



**6- VARIEDAD: clasificador, predicción y evaluación.**

**Al igual que para el sexo, usamos un modelo basado en Random Forest con 500 árboles.**

**Clasificador**

In [15]:
clf_variety = RandomForestClassifier(500).fit(x_train, train['variedad'])

**Predicción**

In [16]:
predicted_variety = clf_variety.predict(x_test)

Ejemplo de la predicción para los 200 últimos usuarios, que pertenecen a la variedad del español de Venezuela.

Usamos para ello la función Counter de la librería collections.

In [17]:
from collections import Counter

c = Counter(predicted_variety[1200:1400])
c.most_common()

[('venezuela', 185),
 ('colombia', 5),
 ('spain', 4),
 ('argentina', 2),
 ('mexico', 2),
 ('peru', 1),
 ('chile', 1)]

**Evaluación**

In [19]:
print(metrics.classification_report(test.variedad, predicted_variety))

             precision    recall  f1-score   support

  argentina       0.93      0.96      0.94       200
      chile       0.97      0.96      0.97       200
   colombia       0.92      0.94      0.93       200
     mexico       0.89      0.93      0.91       200
       peru       0.98      0.88      0.93       200
      spain       0.89      0.95      0.92       200
  venezuela       0.97      0.93      0.95       200

avg / total       0.94      0.94      0.94      1400

